In [26]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

default_algorithm = hashes.SHA256

def hashf(s):
    digest = hashes.Hash(default_algorithm(),backend=default_backend())
    digest.update(s)
    return digest.finalize()


In [ ]:
from cryptography.hazmat.primitives.asymmetric import dh,dsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import *

#Generate some parameters for Diffie-Hellman
parameters_dh = dh.generate_parameters(generator=2,key_size=3072,
                                      backend=default_backend())

#Generate some parameters for DSA
parameters_dsa = dsa.generate_parameters(key_size=3072, 
                                         backend=default_backend())

def DHDSA(conn):
    """
        DH
   Generate Keys
    """
    #generate private key DH
    pk_DH = parameters_dh.generate_private_key()
    #gemerate public key DH
    pub_DH = pk_DH.public_key().public_bytes(encoding=serialization.Encoding.PEM,
                                             format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    
    """
        DSA
   Generate Keys 
    """
    
    #generate private key DSA
    pk_DSA = parameters_dsa.generate_private_key()
    #generate public key DSA
    pub_DSA = pk_DSA.public_key().public_bytes(encoding=serialization.Encoding.PEM,
                                               format=serialization.PublicFormat.SubjectPublicKeyInfo)

    """
    Share Key
    """
    
    #send public key DSA
    conn.send(pub_DSA)
    
    #signing 
    signature = pk_DSA.sign(pub_DH,hashes.SHA256())
    
    peer_pub_DSA = serialization.load_pem_public_key(conn.recv(),
                                                     backend=default_backend())
    
    #send public key DH and signature
    conn.send(pub_DH)
    conn.send(signature)
    
    
    #check if signature is ok
    try:
        peer_pub = conn.recv()
        sig = conn.recv()
        peer_pub_DSA.verify(sig,peer_pub,hashes.SHA256())
        print("ok dsa")
    except InvalidSignature:
        print("fail dsa")
        

    #shared_key calculation
    peer_pub_key = serialization.load_pem_public_key(peer_pub,
                                                    backend = default_backend())
    
    shared_key = pk_DH.exchange(peer_pub_key)
    
    #confirmation
    my_tag = hashf(bytes(shared_key))
    conn.send(my_tag)
    peer_tag = conn.recv()
    if my_tag == peer_tag:
        print('OK DH')
        return shared_key
    else:
        print('FAIL DH')
        
    #erase data
    pk_DH = None
    pub_DH = None
    peer_pub = None
    peer_pub_key = None
    shared_key = None
    my_tag = None
    peer_tag = None  

In [ ]:
from multiprocessing import Pipe, Process

class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vei ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe()
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)
    
    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start()
            self.rproc.start()  
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()
    
    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()

In [24]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def Emitter(conn):
    
    #Acordo de chaves DH e assinatura DSA
    key = DHDSA(conn)
    
    # cria uma mensagem
    mess = bytes('catcatgo!','utf-8') 
    
    # tweak 
    tweak = os.urandom(16)
    
    #Cipher
    cipher = Cipher(algorithms.AES(key), modes.XTS(tweak),
                   backend=default_backend()).encryptor()
    
    conn.send(tweak)
    buffer = bytearray(3) 
    
    
    try:
        while mess.readinto(buffer)
            ciphertext = cipher.update(bytes(buffer))
            conn.send(ciphertext)
        
        conn.send(cipher.finalize())
        
    except Exception as err:
        print("Erro no emissor: {0}".format(err))
        
    
    print(key)
   
    conn.close()
    
    key = None

In [ ]:
def Receiver(conn):
    
    #Acordo de chaves DH e assinatura DSA
    key = DHDSA(conn)
    
    #tweak
    tweak = conn.recv()
    
    #Cifra
    cipher = Cipher(algorithms.AES(key), modes.XTS(tweak),
                   backend = default_backend()).decryptor()
    
    # operar a cifra: ler da conecção um bloco, decifrá-lo e escrever o resultado no 'stream' de output
    try:
        while True:
            try:
                buffer = conn.recv()
                ciphertext = bytes(buffer)
                print(cipher.update(ciphertext))
                if not buffer:
                    print(cipher.finalize())
                    break
            
            except InvalidSignature as err:
                raise Exception("erro autenticação do ciphertext: {}".format(err))
        
    except Exception as err:
        print("Erro no receptor: {0}".format(err))
    

    print(key)
    
    conn.close()

In [ ]:
BiConn(Emitter,Receiver,timeout=30).auto()